# Preprocessing


In [4]:
import pandas as pd

# Load the datasets
rental_price_data = pd.read_csv('../data/rental_price.csv')
rating_features_data = pd.read_csv('../data/rating_features.csv')

print(rental_price_data.head())
print(rating_features_data.head())

(     price
 0  $300.00
 1   $99.00
 2  $100.00
 3  $100.00
 4  $599.00,
    accommodates                                          amenities  \
 0             6  {"Cable TV",Internet,"Wireless Internet","Air ...   
 1             2  {"Air Conditioning",Heating,"Family/Kid Friend...   
 2             2  {TV,"Cable TV",Internet,"Wireless Internet","A...   
 3             2  {TV,"Cable TV",Internet,"Wireless Internet","A...   
 4             6  {TV,"Cable TV",Internet,"Wireless Internet","A...   
 
    availability_30  bathrooms  bed_type  bedrooms  beds  \
 0                0        2.5  Real Bed       1.0   3.0   
 1               29        0.0     Futon       1.0   1.0   
 2               30        1.5  Real Bed       1.0   1.0   
 3               30        1.5  Real Bed       1.0   1.0   
 4               27        2.0  Real Bed       3.0   3.0   
 
    calculated_host_listings_count cancellation_policy  guests_included  ...  \
 0                               1            moderate   